In [2]:
var fs = require('fs');
var zlib = require('zlib');
var path = require('path');
var glob = require('glob');
var strip = require('strip-comments');

function validFileName(s) {
    if(s.match(/node_modules/)) return false;
    if(s.match(/^(lib\dist)/)) return false;
    if(s.match(/\/(try_src|try_it)\//)) return false;
    if(s[0] === '_' || s[0] !== '.') return false;
    if(s.match(/\.test\.js$/)) return false;
    return true;
}

function jsFilesPromise(srcDir, ext='js') {
    if(!srcDir) srcDir = '.';
    //console.log('glob', glob);
    return new Promise((resolve, reject) => {
    	let dir = srcDir+'/**/**.'+ext;
    	//console.log(dir);
        glob(dir, (err,fn) => {
           //console.log(fn,typeof fn);
           if(err) return reject(err);
           
           const res = fn.filter(validFileName);
           let list = res.map(f => valid(f,readLines(f))).filter(r => r !== undefined );
           resolve(list);
        });
    });
}

function readLines(fileName) {
    //console.log("Read from:", fileName);
    return fs.readFileSync(fileName,'utf8').replace(/\r/g, '').split('\n');
}

/**
 * The javascript file has been modified
 * to a header fot tryit at the top
 * Comment section with 'Example' near the ednd
 */
function valid(fileName,lines) {
    let res = parse(fileName,lines);
    
    if(res && res.length > 2) {
      if(res[0].state !== '!md') return undefined;
      let l = res.filter(o => o.state === '!md' && o.data.match('^[ \t]*Example'));
      if(l.length) return [fileName, res];
    }
    return undefined;
}
/**
 * Parse an arroy of line of a javascript source file
 * looking to Note at the top
 * Some javascript code
 *
 */
function parse(fn,lines) {
	//console.log("File:", fn);
	let res = lines.reduce((result, line) => {
		result = result || {};
		let {list, accum, state} = result;
		state = state || 'begin';
		list = list || [];
		accum = accum || [];
        //console.log({state});
		switch(state) {
			case '!md': {
				if(line.trim() === '*/') {
					if(accum.length) {
						if(accum[0].trim().toUpperCase() === 'example') 
							state = 'example';
						list.push({state, data: accum.join('\n')});
					}
					result.list = list; 
					result.accum = undefined;
					result.state = '!tryit';

				}
				else {
					accum.push(line.replace(/^\s*\*\s?/, ''))
					result.accum = accum;
				}
				return result	
			} /*!md*/
			case 'begin': {
				if(line.trim().startsWith('/**')) {
					result.state = '!md';
				}
				return result;
			}
			case '!tryit': {
				if(line.startsWith('/**')) {
					if(accum.length) {
						list.push({state, data: accum.join('\n')});
					}
					result.list = list; 
					result.accum = undefined;
					result.state = '!md';
					return result;
				}
				accum.push(line);
				result.accum = accum;
				return result;
			}
		} 
        return result;
	}, {});
	if(res.list && res.accum && res.accum.length) {
		res.list.push({state: res.state, data: res.accum.join('\n'), file: fn})
	}
   // console.log(JSON.stringigy(res.list));
	return res.list;
}

// Promise.all(jsFilesPromise('.')).then ( list =>
//  list.map(s => console.log(s))
// )

//'1234567890'.replace(/^(\d{3})(\d{3})(\d{4}$)/, "($1) $2-$3")
strip('hello /* how\n are */ you')

'hello  you'

In [36]:
function codeFix(str, moduleName, state) {
    if(state !== '!tryit') return str;
    let ols = str;
    str = str.replace(/main[ \t\n]*\(/gm, moduleName+'Test(');
    str = str.replace(/export[ \t\n]*{[^}]*}/gm, '');
    str = str.replace(/export[ \t\n]+(default)?/gm,'');
    str = str.replace(/['"]use strict['"]/gm,'');
    return str;
}

function getInnerCode(data, start, end,moduleName) {
    let head = '!md' +'\n'+ 
               '# '+moduleName +'\n\n'+
               data[0].data+'\n';
               
    let tail = data.slice(end).map(o => o.state+'\n'+codeFix(o.data, moduleName,o.state)+'\n').join('\n');
    let res = data
               .slice(start, end)
               .filter(o => o.state === '!tryit')
               .map(o => codeFix(o.data, moduleName,o.state));
    if(res.length) {
        return '!head'+`
  <title>${moduleName}</title>
  <script src="/try_it/tryit-helper.js"></script>
${head}
!tryit
${res.join('\n')}
${tail}
!end
<script>
/*
if(!console.logx) {
  console.logx = console.log;
  console.log = function(...args) {
    let res = args.filter(x=>x).map(x => '<pre>'+escapeHTML(x.toString())+'</pre>').join('&nbsp;');
    $$.HTML(res+'<br>');
  }
}
*/
let cc = $$.codeTransform;
cc.add(cc.comment);
cc.add(cc.class);
cc.add(src => src.replace(/console.log/, '$$$$.D'));
cc.add(src => (console.log(src),src)); // display transformed data
</script>
`;
    }
}

function getModuleName(fileName) {
    let list = fileName.split('/');
    //console.log(list);
    return list[list.length-1].replace('.js', '');
}

undefined

In [21]:
function fileProcessor(basePath){
    return ([fileName,data], i) => {

          let target = basePath+'/try_src/'+(
              fileName.startsWith(basePath)? 
                  fileName.substr(basePath.length):
                  fileName
          );
          target = target.replace(/\/+/g, '/').replace(/\.js$/, '.try');
          let moduleName = getModuleName(fileName);
          let start = data[0].data;
          if( data[0].state !== '!md') return undefined;
          let ix = data.findIndex(o => o.state === '!md' && o.data.match(/^[ \t]*Example/));
          if(ix === -1 ) return undefined;
          let targetContent = getInnerCode(data,1,ix, moduleName);
          if(targetContent[0] === '\n') targetContent = targetContent.substr(1);
          console.log('file',fileName,target,ix);
//           if(Math.random() < 0.1) {
//               console.log('@@@'+targetContent);
//           }
          return ({fileName, target, targetContent})
        };
}

function writeOut(fileName,str, makePath=true) {    
  if(makePath) {
    let targetDir = path.dirname(fileName);
    fs.mkdirSync(targetDir, { recursive: true });
  }
  console.log("****   writOut", fileName);
  fs.writeFileSync(fileName, str, 'utf8');
}


undefined

In [26]:
function makeNode(name) {
    return ({name, children: []})
}

function children(aTree) {
    if(!aTree || !aTree.children ) return 0;
    return aTree.children.length;
}

function collectPaths(aTree,base) {
    let b =  base+'/'+aTree.name;
    if(children(aTree) === 0) return b;
    return aTree.children.flatMap(n => collectPaths(n,b));
}

function treeBuild(tree, file) {
    //console.log(tree, file);
    if(tree === undefined) tree = makeNode('..');
    if(Array.isArray(tree)) {
        let node = tree.find(e => e.name === file);
        if(node === undefined) {
            node = makeNode(file);
            tree.push(node);
        }
        return node.children;
    } else {
        return tree.children;
    }
}
function buildIndex(tree, {target}) {
  let arr = target.split('/').slice(1); console.log('PATH', arr);
  arr.reduce(treeBuild,tree);
  //console.log({target}, JSON.stringify(tree,null,' '))
  return tree;
}

//====================================================================

function genIndexFiles(base, list) {
   let tmp = list
         .map(([names, files]) => ({name: names.join('/'), files}))
         .map(({name, files}) => ({name, files: files.map(n => n.substr(name.length+1))}));
    //tmp.forEach(v => console.log('index.html =>',v));
    return tmp;
}



function getNextPrevBack(target, fileList) {
    if(!Array.isArray(fileList)) {
        console.log("NO FILELIST");
        return ['','','../index.html'];
    }
    let match = fileList.find(({name,files}) => fileMatch(files,target) !== -1);
    if(!match) return ['','', '../index.html']
    let {name, files} = match;
    let ix = fileMatch(files,target);
    if(ix === 0) { return [(files[1]||''), '', '../index.html']; }
    if(ix+1 === files.length) return ['',(files[ix-1]||''), '../index.html']; 
    return [files[ix+1], files[ix-1], '../index.html']
}

function fileMatch(list, target) {
    return list.findIndex(name => target.indexOf(name) !== -1);
}

function makeTargetContent({target, targetContent}, fileOrg) {
    let [next, prev, back] = getNextPrevBack(target,fileOrg);
    let tail = '!end\n<script>\nsetNext('+JSON.stringify( [next, prev, back])+");\n</script>\n";
    return targetContent+tail;
}



undefined

In [14]:
// Testing

//JSON.stringify(buildIndex(makeNode('..'), { target: '../try_src/String/GenerateGUID.try'}));
var res = [{ target: '../try_src/String/match.try'}, { target: '../try_src/String/GenerateGUID.try'}, 
          { target: '../try_src/Sorts/bubble.try'}].reduce(buildIndex,makeNode('try_src'));
res.children.map(c => [[res.name, c.name], collectPaths(c, res.name)]);


PATH [ 'try_src', 'String', 'match.try' ]
PATH [ 'try_src', 'String', 'GenerateGUID.try' ]
PATH [ 'try_src', 'Sorts', 'bubble.try' ]


[
  [
    [ 'try_src', 'String' ],
    [ 'try_src/String/match.try', 'try_src/String/GenerateGUID.try' ]
  ],
  [ [ 'try_src', 'Sorts' ], [ 'try_src/Sorts/bubble.try' ] ]
]

In [15]:
// Testing

var testList = [
    { //0
      name: 'try_src/Backtracking',
      files: [ 'KnightTour.try', 'NQueen.try', 'Sudoku.try' ]
    },
    { //1
        name: 'try_src/Cache', files: [ 'LFUCache.try', 'LRUCache.try' ] },
    { //2
      name: 'try_src/Ciphers',
      files: [
        'CaesarsCipher.try',
        'KeyFinder.try',
        'ROT13.try',
        'VigenereCipher.try',
        'XORCipher.try'
      ]
    },
    { //3
      name: 'try_src/Conversions',
      files: [
        'ArbitraryBase.try',
        'BinaryToDecimal.try',
        'DecimalToBinary.try',
        'HexToRGB.try',
        'RGBToHex.try',
        'RomanToDecimal.try'
      ]
    }    
];
fileMatch(testList[2].files, '../try_src/Ciphers/ROT13.try');
makeTargetContent({ target:'../try_src/Ciphers/ROT13.try', targetContent: "HEAD\nBODY\n"}, testList)

'HEAD\n' +
  'BODY\n' +
  '!end\n' +
  '<script>\n' +
  'setNext(["VigenereCipher.try","KeyFinder.try","../index.html"]);\n' +
  '</script>\n'

In [16]:
codeFix(`  "use strict";
   export default  function main 
(n) {
  const board = new NQueen(n)

  board.printBoard()
  console.log('\n')
  export 
{hello, how are
   };
  // hello 
  export const func = () => true;
  export const fred;
`, 'nurul', '!tryit');


'  ;\n' +
  '     function nurulTest(n) {\n' +
  '  const board = new NQueen(n)\n' +
  '\n' +
  '  board.printBoard()\n' +
  "  console.log('\n" +
  "')\n" +
  '  ;\n' +
  '  // hello \n' +
  '  const func = () => true;\n' +
  '  const fred;\n'

In [38]:
function genTyrFiles(base='.') {
    let data = fs.readFileSync(base+'/_scripts/tryit-helper.js','utf8').replace(/\r/g, '');
    writeOut(base+'/tryit_src/tryit-helper.js',data);
    jsFilesPromise(base+'/').then(list =>{
      return list.map(fileProcessor(base)).filter(x => x!==undefined);
    }).then(list => {
       // list.forEach(o => writeOut(o.target, o.targetContent));
        return list
     }).then(list => [list.reduce(buildIndex, makeNode('try_src')), list]
     ).then(([tree, list]) => [
         tree.children.map(c => [[tree.name, c.name], collectPaths(c, tree.name)]),
         list
       ]
     ).then(([res,list]) => ([genIndexFiles(base,res), list])
     ).then(([res,list]) => {
        list.forEach( o => writeOut(o.target, makeTargetContent(o, res)))   
      });
 }
genTyrFiles('..')
'done'

'done'

file ../Backtracking/KnightTour.js ../try_src/Backtracking/KnightTour.try 2
file ../Backtracking/NQueen.js ../try_src/Backtracking/NQueen.try 2
file ../Backtracking/Sudoku.js ../try_src/Backtracking/Sudoku.try 2
file ../Cache/LFUCache.js ../try_src/Cache/LFUCache.try 2
file ../Cache/LRUCache.js ../try_src/Cache/LRUCache.try 2
file ../Ciphers/CaesarsCipher.js ../try_src/Ciphers/CaesarsCipher.try 4
file ../Ciphers/KeyFinder.js ../try_src/Ciphers/KeyFinder.try 2
file ../Ciphers/ROT13.js ../try_src/Ciphers/ROT13.try 4
file ../Ciphers/VigenereCipher.js ../try_src/Ciphers/VigenereCipher.try 10
file ../Ciphers/XORCipher.js ../try_src/Ciphers/XORCipher.try 4
file ../Conversions/ArbitraryBase.js ../try_src/Conversions/ArbitraryBase.try 4
file ../Conversions/BinaryToDecimal.js ../try_src/Conversions/BinaryToDecimal.try 6
file ../Conversions/DecimalToBinary.js ../try_src/Conversions/DecimalToBinary.try 2
file ../Conversions/HexToRGB.js ../try_src/Conversions/HexToRGB.try 2
file ../Conversions/RGB

PATH [ 'try_src', 'Sorts', 'BeadSort.try' ]
PATH [ 'try_src', 'Sorts', 'BogoSort.try' ]
PATH [ 'try_src', 'Sorts', 'BubbleSort.try' ]
PATH [ 'try_src', 'Sorts', 'BucketSort.try' ]
PATH [ 'try_src', 'Sorts', 'CocktailShakerSort.try' ]
PATH [ 'try_src', 'Sorts', 'CombSort.try' ]
PATH [ 'try_src', 'Sorts', 'CountingSort.try' ]
PATH [ 'try_src', 'String', 'CheckRearrangePalindrome.try' ]
PATH [ 'try_src', 'String', 'CheckVowels.try' ]
PATH [ 'try_src', 'String', 'createPurmutations.try' ]
PATH [ 'try_src', 'String', 'FormatPhoneNumber.try' ]
PATH [ 'try_src', 'String', 'GenerateGUID.try' ]
PATH [ 'try_src', 'String', 'KMPPatternSearching.try' ]
PATH [ 'try_src', 'String', 'MaxCharacter.try' ]
PATH [ 'try_src', 'String', 'PermutateString.try' ]
PATH [ 'try_src', 'String', 'ReverseWords.try' ]
PATH [ 'try_src', 'String', 'ValidateEmail.try' ]
PATH [ 'try_src', 'Timing-Functions', 'GetMonthDays.try' ]
PATH [ 'try_src', 'Trees', 'BreadthFirstTreeTraversal.try' ]
PATH [ 'try_src', 'Trees', 'Dep

In [ ]:
glob('../Sorts/**/**.js', (err,fn) => {
           console.log(fn,typeof fn);
       });

In [ ]:
function F(n) {
   let m = new Map(); /// memoize intermediate results
   m.set(0,0n);  // F0
   m.set(1,1n);  // F1
   m.set(2,1n);  // F2
   return  f(n);
   function f(n) {
     let r;
     if( (r= m.get(n)) !== undefined ) return r;
     if( !(n&1) ) {
        let x = n >>1;
	let fx = f(x);
	let fx1 = f(x-1);    
        r = fx*(fx+ fx1 + fx1);
     } else {
        let x = (n+1) >> 1;
	let fx = f(x);
	let fx1 = f(x-1);
	r = fx*fx + fx1*fx1
     }
     m.set(n,r);
     return r;
    }
 }

In [100]:
function rotateListRightInPlace (
          head,
          k, 
          next = head => head.next, 
          setNext=(node, val) => node?(node.next = val):undefined 
          ) {
  if(!head) return head;
  let len = 0
  let current = head;
  let tail;
  k
  while (current) {
    tail = current;
    current = next(current);
    len++
  }
  if(len === 1) return head;
  if(k>0) k = len-k;
  if(k<0) k = -k;
  k %= len;

  while(k--) {
    let current = head;
    [head, tail] = [next(current), setNext(tail,current)];
  }  
  setNext(tail,undefined);
  return head
}

/**
 * Example
 */

//Create a toString() method for a list node
function listToString(node) {
  if(!node) return 'null';
  return node.val + ' => '+ listToString(node.next);
}

/**
 * ## Test Rotation
 */

// Functio to create a linked list - for these tests
var list = () => [1,2,3,4,5,6,7]
    .reverse()
    .reduce((head,el) => 
        ({val: el, next: head, toString: listToString}), 
        undefined
    );

/**
 * Now test the code
 */

function rotate(n) {
  console.log('Rotate  '+n+': ', listToString(rotateListRightInPlace(list(),n)));    
}

// set's see some results              
[0,1,4,6,7,9,-1, -2].forEach(rotate);


Rotate  0:  1 => 2 => 3 => 4 => 5 => 6 => 7 => null
Rotate  1:  7 => 1 => 2 => 3 => 4 => 5 => 6 => null
Rotate  4:  4 => 5 => 6 => 7 => 1 => 2 => 3 => null
Rotate  6:  2 => 3 => 4 => 5 => 6 => 7 => 1 => null
Rotate  7:  1 => 2 => 3 => 4 => 5 => 6 => 7 => null
Rotate  9:  3 => 4 => 5 => 6 => 7 => 1 => 2 => null
Rotate  -1:  2 => 3 => 4 => 5 => 6 => 7 => 1 => null
Rotate  -2:  3 => 4 => 5 => 6 => 7 => 1 => 2 => null


undefined

In [ ]:
/**
 * Implementation of a doubly linked list
 *
 * Hamza chabchoub contribution for a university project
 */


// 
function DoubleLinkedList () {
  const Node = function (element) {
    this.element = element
    this.next = null
    this.prev = null
  }
  Node.prototype.toString = function() {
    return ''+this.element;
  }

  let length = 0
  let head = null
  let tail = null

  this.newNode = function(element) {
    return new Node(element);
  }
  // Add new element
  this.append = function (element) {
    const node = new Node(element)

    if (!head) {
      head = node
      tail = node
    } else {
      node.prev = tail
      tail.next = node
      tail = node
    }

    length++
  }

  // Add element
  this.insert = function (position, element) {
    // Check of out-of-bound values
    if (position >= 0 && position <= length) {
      const node = new Node(element)
      let current = head
      let previous = 0
      let index = 0

      if (position === 0) {
        if (!head) {
          head = node
          tail = node
        } else {
          node.next = current
          current.prev = node
          head = node
        }
      } else if (position === length) {
        current = tail
        current.next = node
        node.prev = current
        tail = node
      } else {
        while (index++ < position) {
          previous = current
          current = current.next
        }

        node.next = current
        previous.next = node

        // New
        current.prev = node
        node.prev = previous
      }

      length++
      return true
    } else {
      return false
    }
  }

  // Remove element at any position
  this.removeAt = function (position) {
    // look for out-of-bounds value
    if (position > -1 && position < length) {
      let current = head
      let previous = 0
      let index = 0

      // Removing first item
      if (position === 0) {
        head = current.next

        // if there is only one item, update tail //NEW
        if (length === 1) {
          tail = null
        } else {
          head.prev = null
        }
      } else if (position === length - 1) {
        current = tail
        tail = current.prev
        tail.next = null
      } else {
        while (index++ < position) {
          previous = current
          current = current.next
        }

        // link previous with current's next - skip it
        previous.next = current.next
        current.next.prev = previous
      }

      length--
      return current.element
    } else {
      return null
    }
  }

  // Get the indexOf item
  this.indexOf = function (elm) {
    let current = head
    let index = -1

    // If element found then return its position
    while (current) {
      if (elm === current.element) {
        return ++index
      }

      index++
      current = current.next
    }

    // Else return -1
    return -1
  }

  // Find the item in the list
  this.isPresent = (elm) => {
    return this.indexOf(elm) !== -1
  }

  // Delete an item from the list
  this.delete = (elm) => {
    return this.removeAt(this.indexOf(elm))
  }

  // Delete first item from the list
  this.deleteHead = function () {
    this.removeAt(0)
  }

  // Delete last item from the list
  this.deleteTail = function () {
    this.removeAt(length - 1)
  }

  // Print item of the string
  this.toString = function () {
    let current = head
    let string = ''

    while (current) {
      string += current.element + (current.next ? ', ' : '')
      current = current.next
    }

    return '( '+string+' )'
  }

  // Convert list to array
  this.toArray = function () {
    const arr = []
    let current = head

    while (current) {
      arr.push(current.element)
      current = current.next
    }

    return arr
  }

  // Check if list is empty
  this.isEmpty = function () {
    return length === 0
  }

  // Get the size of the list
  this.size = function () {
    return length
  }

  // Get the head
  this.getHead = function () {
    return head
  }

  // Get the tail
  this.getTail = function () {
    return tail
  }
}

/**
 * Example
 */

var newDoubleLinkedList = new DoubleLinkedList();
console.log('isEmpty: ' + newDoubleLinkedList.isEmpty(), 'Expect true'); 
[1,2,3,4,5,6].forEach(e => newDoubleLinkedList.append(e));

console.log('display:( ' + newDoubleLinkedList.toString(), ') Expect ( 1, 2, 3, 4, 5, 6)');
var aNode = newDoubleLinkedList.newNode(7);
console.log('New Node', aNode.toString(), 'Expect 7'); 

console.log('size: ' + newDoubleLinkedList.size(), 'Expect 6');
newDoubleLinkedList.insert(6,aNode);
console.log('size: ' + newDoubleLinkedList.size(), 'Expect 7 after insert of aNode(7)'); 
console.log('display:( ' + newDoubleLinkedList.toString(), ') Expect ( 1, 2, 3, 4, 5, 6, 7)');
 
console.log('getHead: ' + newDoubleLinkedList.getHead().toString(), 'Expect 1'); 
console.log('getTail: ' + newDoubleLinkedList.getTail().toString(), 'Expect 1'); 
console.log('isEmpty: ' + newDoubleLinkedList.isEmpty(), 'Expect false'); 


In [137]:
var Node = class {
    constructor(value) {
        this.value = value;
        this.previous = undefined;
        this.next = undefined;
    }
    toString() { return ''+this.value; }
};

var DList = class {
    constructor() {
        this._head = new Node();
        this._tail = new Node();
        this.head.next = this._tail;
        this.tail.previous = this._head;
        this._length = 0;
        this.lastIndex = -1;
        this.lastNode = undefined;
    }
    
    _newNode(value) {
        return new Node(value);
    }
    get length() { return this._length; }
    get head() { return this._head; }
    get tail() { return this._tail; }
    _indexOf(aNode, startNode) {
        if(this.lastNode && (startNode === this.lastNode || startNode === this.lastNode.next )) {
            if(this.lastNode === aNode) return this.lastIndex;
            if(this.lastNode.next === aNode) {
                this.lastIndex++;
                this.lastNode = aNode;
                return this.lastIndex
            } else if(this.lastNode.previous === aNode) {
                this.lastIndex--;
                this.lastNode = aNode;
                return this.lastIndex;
            }
        }
        let curr = startNode || this._head.next;
        let ix = 0;
        while( curr && curr !== aNode) ix++;
        if(!curr) return -1;
        if(!startNode) {
            this.lastNode = aNode;
            this.lastIndex = ix;
        } else if(startNode === this.lastNode) {
            this.lastNode = aNode;
            this.lastIndex += ix;
        }
        else if(startNode === this.lastNode.next) {
            this.lastNode = aNode;
            this.lastIndex += (ix-1);
        }
        return ix;
    }
    
    _getAt(ix) {
        if(ix >= this.length || ix < 0) return undefined;
        if(this.lastIndex === ix && this.lastNode) return this.lastNode;
        let cur;
        
        if( this.lastNode ) {
            // Check if it is more efficient to starting from `lastNode`
            if(ix < this.lastIndex && ix > (this.lastIndex - ix) ) {
                cur = this.lastNode.previous;
                for(let i=this.lastIndex-1; i > ix; i--) cur = cur.previous;
                this.lastIndex = ix;
                this.lastNode = cur;
                return cur;
            }
            else if( ix > this.lastIndex && ix-this.lastIndex < this.length-ix) {
               cur = this.lastNode.next;
               for(let i=this.lastIndex+1; i< ix; i++) cur = cur.next;
               this.lastIndex = ix;
               this.lastNode = cur;
               return cur;
            }
        } 
        
        // Check if we search from head or from tail.
        if(ix <= this.length-ix) {
           cur = this.head.next;
           for(let i=0; i< ix; i++) cur = cur.next;
        }
        else {
            cur = this.tail.previous;
            for(let i=this.length-1; i > ix; i--) cur = cur.previous;
        }
        this.lastIndex = ix;
        this.lastNode = cur;
        return cur;
    }
    
    _delete(aNode) {
        if(!aNode) return aNode
        aNode.next.previous = aNode.previous;
        aNode.previous.next = aNode.next;
        if(this.lastNode === aNode) {
            if(this.lastNode.previous !== this.head) {
                this.lastNode = aNode.previous;
                this.lastIndex--;
            }
        } else {
            this.lastIndex = -1;
            this.lastNode = undefined;
        }
        this._length--;
        return aNode;
    }
    
    _insertNode(posNode, aNode) {
        aNode.next = posNode.next;
        aNode.previous = posNode.next.previous;
        posNode.next.previous = aNode;
        posNode.next = aNode;
        this._length++;
        if(this.lastNode != posNode) {
            this.lastIndex = -1;
            this.lastNode = undefined;
        }
        return aNode;
    }
    indexOf(Value,startNode) {
        let curr = startNode || this.head.next;
        let ix = 0;
        while( curr && curr.value !== value) ix++;
        if(!curr) return -1;
        this.lastNode = cur;
        this.lastIndex = ix;
        return ix;
    }
    
    findNode(aValue, startNode) {
        let ix = this.indexOf(aValue, startNode);
        if( ix === -1 ) return -1;
        return this._getAt(ix);
    }
    append(value) {
        let aNode = new Node(value);
        this._insertNode(this.tail.previous, aNode);
        //this._length++;
        return this;
    }
    
    isLast(aNode) {
        return aNode === this._tail || this.tail.previous === aNode;
    }
    
    toString() {
        let list = [];
        let cur = this.head.next;
        for(; !this.isLast(cur); cur = cur.next) list.push(''+cur);
        return '['+list.join(',')+']';
    }
}

undefined

In [147]:
var dlist = new DList();
var append = v => dlist.append(v);
function range(n) { let list = []; for(let i=0; i<n; i++) list.push(i); return list;}
range(5000).forEach(append);
console.log(dlist.length);
for(let i=0; i<10_000_000; i++) {dlist._getAt(30); dlist._getAt(32); dlist._getAt(260); }

5000


Node {
  value: 260,
  previous: Node {
    value: 259,
    previous: Node { value: 258, previous: [Node], next: [Circular] },
    next: [Circular]
  },
  next: Node {
    value: 261,
    previous: [Circular],
    next: Node { value: 262, previous: [Circular], next: [Node] }
  }
}